首先我们使用tetgen来测试一下三角形到四面体的转换

In [1]:
import pyvista as pv
import tetgen
import numpy as np
import openmesh as om
from openmesh import TriMesh
from pyvista import PolyData


In [ ]:
sphere = pv.Sphere()
tet = tetgen.TetGen(sphere)
tet.tetrahedralize(order=1, mindihedral=20, minratio=1.5)
grid = tet.grid

# get cell centroids
cells = grid.cells.reshape(-1, 5)[:, 1:]
cell_center = grid.points[cells].mean(1)

# extract cells below the 0 xy plane
mask = cell_center[:, 2] < 0
cell_ind = mask.nonzero()[0]
subgrid = grid.extract_cells(cell_ind)

# advanced plotting
plotter = pv.Plotter()
plotter.add_mesh(subgrid, 'lightgrey', lighting=True, show_edges=True)
plotter.add_mesh(sphere, 'r', 'wireframe')
plotter.add_legend([[' Input Mesh ', 'r'],
                    [' Tessellated Mesh ', 'black']])
plotter.show()

In [6]:
class Mesh:
    def __init__(self, model_path='./models/255.obj'):
        self.mesh:TriMesh = om.read_trimesh(model_path)
    
    def update_v_f(self):
        v = [self.mesh.point(vertex).tolist() for vertex in self.mesh.vertices()]
        f = [[vh.idx() for vh in self.mesh.fv(face)] for face in self.mesh.faces()]
        self.f, self.v = f, v
    
    def pv_mesh(self) -> PolyData:
        self.update_v_f()
        '''
        https://docs.pyvista.org/version/stable/examples/00-load/create-poly.html#ref-create-poly
        '''
        vertices = np.array(self.v)
        constant = np.full((len(self.f), 1), len(self.f[0])) # assuming all faces have same vertex num 
        faces = np.hstack((constant, self.f))
        faces = np.hstack(faces) # notice faces is 1-D array
        pyvista_mesh = pv.PolyData(vertices, faces)
        return pyvista_mesh

def test():
    pv.set_plot_theme('document')

    sphere = pv.Sphere()
    tet = tetgen.TetGen(sphere)
    tet.tetrahedralize(order=1, mindihedral=20, minratio=1.5)
    grid = tet.grid
    grid.plot(show_edges=True)

def test2():
    pv.set_plot_theme('document')

    a = Mesh(model_path='./models/Gargoyle_ABF.obj')
    b = a.pv_mesh()
    tet = tetgen.TetGen(b)
    tet.tetrahedralize(order=1, mindihedral=20, minratio=1.5)
    grid = tet.grid
    # grid.plot(show_edges=True)

    # get cell centroids
    cells = grid.cells.reshape(-1, 5)[:, 1:]
    cell_center = grid.points[cells].mean(1)

    # extract cells below the 0 xy plane
    mask = cell_center[:, 2] > 0
    cell_ind = mask.nonzero()[0]
    subgrid = grid.extract_cells(cell_ind)

    # advanced plotting
    plotter = pv.Plotter()
    plotter.add_mesh(subgrid, 'lightgrey', lighting=True, show_edges=True)
    plotter.add_mesh(b, 'r', 'wireframe')
    plotter.add_legend([[' Input Mesh ', 'r'],
                        [' Tessellated Mesh ', 'black']])
    plotter.show()


    
    




In [7]:
#test()
test2()



Widget(value="<iframe src='http://localhost:62975/index.html?ui=P_0x165371bd2b0_1&reconnect=auto' style='width…